## Plot Transit Routes with Low Speeds / High Trips 

Plot the transit route's line geom. Not highway segments. See what shows up.

In [1]:
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import Markdown

from bus_service_utils import (calenviroscreen_lehd_utils, 
                               create_parallel_corridors)
from shared_utils import calitp_color_palette as cp
from shared_utils import geography_utils

catalog = intake.open_catalog("./*.yml")

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
def prep_data_for_viz():
    gdf = catalog.bus_routes_aggregated_stats.read()
    gdf = gdf.assign(
        route_length = gdf.geometry.to_crs(geography_utils.CA_StatePlane).length
    )
    # Fix types, rounding
    drop_cols = ["route_length", "bus_difference_mean", 
                 "bus_multiplier_mean", "bus_difference_spread"]
    
    gdf = gdf.assign(
        route_length_mi = round(gdf.route_length / geography_utils.FEET_PER_MI, 2),
        mean_speed_mph = gdf.mean_speed_mph.round(1),
        pct_trips_competitive = gdf.pct_trips_competitive.round(2)
    ).drop(columns = drop_cols).to_crs(geography_utils.WGS84)
    
    integrify_me = ["num_competitive", "trips_all_day", "trips_peak"]
    gdf[integrify_me] = gdf[integrify_me].astype("Int64")
    
    # Missing values present, drop now
    gdf = gdf[gdf.caltrans_district.notna()].reset_index(drop=True)
    
    return gdf

gdf = prep_data_for_viz()

In [4]:
def make_map(gdf: gpd.GeoDataFrame, district: str,
             trips_col: str, speed_col: str) -> gpd.GeoDataFrame: 
    
    
    table_title = (f"Routes in top 50% of {trips_col.replace('_', ' ')} and "
                   f"bottom 50% of {speed_col.replace('_', ' ')}"
                  )
    
    display(Markdown(f"#### {table_title}"))
    display(gdf)
    
    m = gdf.drop(columns = "service_date").explore(
        "route_id", categorical=True,
        cmap = cp.CALITP_CATEGORY_BOLD_COLORS,
        tiles = "Carto DB Positron"
    )
    
    display(Markdown(f"### District {district} Routes with Low Speeds / High Trips"))
    display(m)


def identify_high_trip_low_speeds_by_district(df: gpd.GeoDataFrame, 
                                              district: str, trips_col: str, 
                                              speed_col: str) -> gpd.GeoDataFrame:
    subset = df[df.caltrans_district == district].reset_index(drop=True)
    
    print(f"# routes: {len(subset)}")
    
    stats_cols = ["trips_peak", "trips_all_day", 
                  "mean_speed_mph", "pct_trips_competitive"]
    
    subset2 = subset[(subset.mean_speed_mph.notna()) & 
                     (subset[trips_col].notna()) & 
                     (subset.geometry.notna())
                    ].reset_index(drop=True)
    
    if len(subset2) > 0:
        print(f"# routes with full info:{len(subset2)}")
        print(f"# routes with missing info: {len(subset) - len(subset2)}")
        
        subset3 = calenviroscreen_lehd_utils.define_equity_groups(
            subset2, percentile_col = stats_cols , num_groups = 4)
        
        # Plot trips that are top 50%, speeds are bottom 50%
        can_be_plotted = subset3[(subset3[f"{trips_col}_group"] >= 3) & 
                                 (subset3[f"{speed_col}_group"] <= 2)
                                ]
        if len(can_be_plotted) > 0:
            make_map(can_be_plotted, district, trips_col, speed_col)

            return subset3
        else:
            print(f"No observations with combination: top 50% of {trips_col}, bottom 50% of {speed_col}")
    else:
        print("No observations after missing info dropped")

In [5]:
districts = gdf[gdf.caltrans_district.notna()].caltrans_district.unique().tolist()

for d in sorted(districts):
    display(Markdown(f"## District {d}"))
    
    district = identify_high_trip_low_speeds_by_district(
        gdf, d, 
        trips_col = "trips_all_day", speed_col = "mean_speed_mph")

## District 01 - Eureka

# routes: 16
# routes with full info:4
# routes with missing info: 12


#### Routes in top 50% of trips all day and bottom 50% of mean speed mph

,calitp_itp_id,route_id,geometry,service_date,route_type,route_group,num_competitive,caltrans_district,route_length_mi,trips_peak,trips_peak_group,trips_all_day,trips_all_day_group,mean_speed_mph,mean_speed_mph_group,pct_trips_competitive,pct_trips_competitive_group
0,159,2017,"LINESTRING (-122.61575 38.92995, -122.61534 38...",2022-05-04,3,medium,0,01 - Eureka,37.14,14,4,21,4,28.1,1,0.0,1


### District 01 - Eureka Routes with Low Speeds / High Trips

...

### District 10 - Stockton Routes with Low Speeds / High Trips

## District 11 - San Diego

# routes: 18
No observations after missing info dropped


## District 12 - Irvine

# routes: 6
No observations after missing info dropped
